In [4]:
#pip install opencv-python
#pip install chrono
#pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [38]:
import shutil
import face_recognition
import cv2
import numpy as np
import glob
import os
import logging
from datetime import datetime,timedelta
import time
import datetime
from PIL import Image
from chrono import Timer,time
import pandas as pd

In [39]:
wkngdir = os.getcwd()
folder = wkngdir+"/Results"
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [40]:
os.chdir(folder)
persons_dir = "uniquepersons"
video_out_dir = "videoByPerson"
os.makedirs(video_out_dir,exist_ok=True)
os.makedirs(persons_dir,exist_ok=True)

os.chdir(wkngdir)
persons_dir = "Results//"+persons_dir
video_out_dir = "Results//"+video_out_dir

video_capture = cv2.VideoCapture(0)
data = pd.DataFrame(columns=['person','framecount'])

count=0
video_array = []

In [41]:
def checktheframe(face_encodings,persons_dir):
    matched = False
    iter_t = 0
    for filename in os.listdir(persons_dir):
        path = os.path.join(persons_dir,filename)
        filter_image = face_recognition.load_image_file(path)
        filter_face_encoding = face_recognition.face_encodings(filter_image)
        known_face_encodings.append(filter_face_encoding[0])
        known_face_names.append(filename)
        matches = face_recognition.compare_faces(known_face_encodings, face_encodings[0])
        if True in matches:
            matched = True
            data['framecount'] += 1
            video_array[iter_t].write(frame)
            break;
        iter_t = iter_t+1;       
    return matched

In [42]:
while True:
    ret, frame = video_capture.read()
    process_this_frame = True
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        video_capture.release();
        print("Escape hit, closing...") 
        break
    if not (ret):
        logging.error("Could not read frame from camera. Stopping video capture.")
        break
   
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]

    height, width, layers = frame.shape
    size = (width,height)
    if process_this_frame:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        if len(face_encodings) > 0:
            matched = checktheframe(face_encodings,persons_dir)
            if matched == False:
                cv2.imwrite(os.path.join(persons_dir,"person_"+str(count)+".jpg"),frame)
                data = data.append({'person':os.path.join(persons_dir,"person_"+str(count)),'framecount':1},ignore_index=True)  
                temp_video = cv2.VideoWriter(os.path.join(video_out_dir,"videoex_"+str(count)+".avi"),
                                             cv2.VideoWriter_fourcc(*'DIVX'),15,size,isColor = True)
                video_array.append(temp_video)
                count+=1
            
          
cv2.destroyAllWindows();
data.to_excel("VideoAnalysis1.xlsx")
for t in video_array:
    t.release()
            
print(data)
print(video_array)


Escape hit, closing...
                            person framecount
0  Results//uniquepersons\person_0         24
[<VideoWriter 000002558BD5F5F0>]


In [26]:
video_capture.release();
cv2.destroyAllWindows();